In [2]:
import numpy as np
import ase.data

In [3]:
atomic_masses = { k: ase.data.atomic_masses[v] 
                 for k,v in ase.data.atomic_numbers.items() }
print(atomic_masses)

{'X': 1.0, 'H': 1.008, 'He': 4.002602, 'Li': 6.94, 'Be': 9.0121831, 'B': 10.81, 'C': 12.011, 'N': 14.007, 'O': 15.999, 'F': 18.998403163, 'Ne': 20.1797, 'Na': 22.98976928, 'Mg': 24.305, 'Al': 26.9815385, 'Si': 28.085, 'P': 30.973761998, 'S': 32.06, 'Cl': 35.45, 'Ar': 39.948, 'K': 39.0983, 'Ca': 40.078, 'Sc': 44.955908, 'Ti': 47.867, 'V': 50.9415, 'Cr': 51.9961, 'Mn': 54.938044, 'Fe': 55.845, 'Co': 58.933194, 'Ni': 58.6934, 'Cu': 63.546, 'Zn': 65.38, 'Ga': 69.723, 'Ge': 72.63, 'As': 74.921595, 'Se': 78.971, 'Br': 79.904, 'Kr': 83.798, 'Rb': 85.4678, 'Sr': 87.62, 'Y': 88.90584, 'Zr': 91.224, 'Nb': 92.90637, 'Mo': 95.95, 'Tc': 97.90721, 'Ru': 101.07, 'Rh': 102.9055, 'Pd': 106.42, 'Ag': 107.8682, 'Cd': 112.414, 'In': 114.818, 'Sn': 118.71, 'Sb': 121.76, 'Te': 127.6, 'I': 126.90447, 'Xe': 131.293, 'Cs': 132.90545196, 'Ba': 137.327, 'La': 138.90547, 'Ce': 140.116, 'Pr': 140.90766, 'Nd': 144.242, 'Pm': 144.91276, 'Sm': 150.36, 'Eu': 151.964, 'Gd': 157.25, 'Tb': 158.92535, 'Dy': 162.5, 'Ho': 1

In [4]:
class Atom:
    def __init__(self, symbol, x, y, z):
        self.symbol = symbol
        self.position = np.array([x, y, z])
        self.velocity = np.zeros(3)
        self.mass = atomic_masses[symbol]
    def __repr__(self):
        template = "Atom(symbol='{}', position=({:.3f}, {:.3f}, {:.3f}) )"
        return template.format(self.symbol, 
                               self.position[0],
                               self.position[1],
                               self.position[2])

In [5]:
class Molecule(list):     ## We inherit from built-in class list
    # __init__ has been removed.  We could extend it with a type-check
    def get_positions(self):
        "Return the positions of all atoms as an Nx3 array"
        return np.array([a.position for a in self])
    def set_positions(self, pos):
        "Set the postions of all atoms"
        assert len(pos) == len(self)
        for i, p in enumerate(pos):
            self[i].position = p
    def get_symbols(self):
        "Return a list of the chemical symbols of all atoms"
        return [a.symbol for a in self]
    def get_velocities(self):
        "Return the velocities of all atoms as an Nx3 array"
        return np.array([a.velocity for a in self])
    def set_velocities(self, vel):
        "Set the postions of all atoms"
        assert len(vel) == len(self)
        for i, v in enumerate(vel):
            self[i].velocity = v
    def get_masses(self):
        "Get the mass of all atoms"
        return np.array([a.mass for a in self])
    def __repr__(self):
        "The representation - for brevity include only symbols"
        template = "Atoms(N={}, symbols: {})"
        all_symbols = " ".join(self.get_symbols())
        return template.format(len(self), all_symbols)
    def write_to_file(self, filename):
        f = open(filename, "at")
        print(len(self), file=f)
        print("A molecule", file=f)
        template = "{}  {:.3f} {:.3f} {:.3f}"
        for a in self:
            print(template.format(a.symbol, a.position[0], a.position[1], a.position[2]), file=f)
        f.close()


In [6]:
class LennardJones:
    """Lennard Jones potential for a single element."""
    def __init__(self, element, epsilon, sigma):
        """Create the potential.
        
        element: The symbol of the element that will be supported.
        
        epsilon and sigma: The LJ parameters
        """
        self.element = element
        self.sigma = sigma
        self.epsilon = epsilon
        
    def calculate(self, system):
        # Check that all atoms have the right element
        for a in system:
            if a.symbol != self.element:
                raise ValueError("Found element {} but only support {}".format(
                    a.symbol, self.element))
        # The calculation could be optimized by calculating energy and
        # forces in the same loop, but keep is simple to begin with...
        E = self.calculate_energy(system)
        F = self.calculate_forces(system)
        return E, F
    
    def calculate_energy(self, system):
        U = 0
        N = len(system)  # Number of atoms
        for i in range(N):
            for j in range(i+1, N):
                pos_i = system[i].position
                pos_j = system[j].position
                dij = self.calculate_d_ij(pos_i, pos_j)
                U += self.calculate_U(dij)
        return U
    
    def calculate_d_ij(self, pos_i, pos_j):
        d = pos_i - pos_j
        d_squared = d * d
        return np.sqrt(d_squared.sum())
    
    def calculate_U(self, dij):
        x = (self.sigma / dij)**6
        return 4 * self.epsilon * (x**2 - x)
    
    def calculate_forces(self, system):
        # This is NOT EFFICIENT.  Priority has been given to stay
        # close to the equations.
        N = len(system)  # Number of atoms
        F = np.zeros((N, 3))
        for k in range(N):
            pos_k = system[k].position
            for i in range(N):
                for j in range(i+1, N):
                    # Only nonzero if i = k or j = k.  Skip already here to save time
                    if k == i or k == j:
                        pos_i = system[i].position
                        pos_j = system[j].position
                        dij = self.calculate_d_ij(pos_i, pos_j)
                        dU_ddij = self.calculate_dUddij(dij)
                        # We are calculating components for all three alpha together
                        if k == i:
                            ddij_dr = self.calculate_ddr(pos_k, pos_j)
                        else:  # k == j
                            ddij_dr = self.calculate_ddr(pos_k, pos_i)
                        F[k] += - dU_ddij * ddij_dr
        return F
    
    def calculate_dUddij(self, dij):
        x = (self.sigma / dij)**6
        return 4 * self.epsilon * (- 12 * x**2 / dij + 6 * x / dij)
    
    def calculate_ddr(self, p1, p2):
        return (p1 - p2) / self.calculate_d_ij(p1, p2)    

The Velocity Verlet algorithm.


In [10]:
class VelocityVerlet:
    def __init__(self, atoms, model, timestep):
        self.atoms = atoms
        self.model = model
        self.timestep = timestep
    def run(self, steps):
        masses = self.atoms.get_masses()
        # masses are now a N array.  We will divide the Nx3 forces with
        # the masses, we need to replicate the masses as a Nx3 array
        masses = np.outer(masses, np.ones(3))
        assert masses.shape == (len(self.atoms),3)
        vel = self.atoms.get_velocities()
        pos = self.atoms.get_positions()
        energy, forces = self.model.calculate(self.atoms)
        for step in range(steps):
            vel += 0.5 * self.timestep * forces / masses
            pos += self.timestep * vel
            self.atoms.set_positions(pos)
            energy, forces = self.model.calculate(self.atoms)
            vel += 0.5 * self.timestep * forces / masses
            self.atoms.set_velocities(vel)

In [11]:
a = np.arange(10)
b = np.ones(3)
np.outer(a, b)

array([[0., 0., 0.],
       [1., 1., 1.],
       [2., 2., 2.],
       [3., 3., 3.],
       [4., 4., 4.],
       [5., 5., 5.],
       [6., 6., 6.],
       [7., 7., 7.],
       [8., 8., 8.],
       [9., 9., 9.]])

In [12]:
def dimer(distance):
    atom1 = Atom('H', 0, 0, 0)
    atom2 = Atom('H', distance, 0, 0)
    return Molecule([atom1, atom2])

epsilon = 4.4778900
sigma = 0.5523570
model = LennardJones('H', epsilon, sigma)

In [20]:
system = dimer(2 * sigma)
print(system[0].position)
dyn = VelocityVerlet(system, model, 0.01)

[0 0 0]


In [14]:
dyn.run(2)
print(system[0].position)

[0.0002923 0.        0.       ]


In [15]:
system = dimer(2 * sigma)
dyn = VelocityVerlet(system, model, 0.01)
for i in range(100):
    dyn.run(5)
    system.write_to_file("moldyn.xyz")

In [16]:
!ase gui moldyn.xyz

DEPRECATION WARNING: The system version of Tk is deprecated and may be removed in a future release. Please don't rely on it. Set TK_SILENCE_DEPRECATION=1 to suppress this warning.
Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.0_2/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 1885, in __call__
    return self.func(*args)
  File "/Users/schiotz/development/ase/ase/gui/ui.py", line 210, in <lambda>
    self.widget.bind('<Return>', lambda event: self.callback())
TypeError: 'NoneType' object is not callable


In [29]:
import os
os.remove('moldyn.xyz')

In [30]:
system = dimer(1.5 * sigma)
atom3 = Atom('H', 0, 0.8, 0)
system.append(atom3)
system.write_to_file("test.xyz")
dyn = VelocityVerlet(system, model, 0.01)

In [31]:
!ase gui test.xyz

DEPRECATION WARNING: The system version of Tk is deprecated and may be removed in a future release. Please don't rely on it. Set TK_SILENCE_DEPRECATION=1 to suppress this warning.


In [32]:
for i in range(100):
    dyn.run(5)
    system.write_to_file("moldyn.xyz")

In [33]:
!ase gui moldyn.xyz

DEPRECATION WARNING: The system version of Tk is deprecated and may be removed in a future release. Please don't rely on it. Set TK_SILENCE_DEPRECATION=1 to suppress this warning.
